<h1>Get all mesh_terms from the Pubmed </h1>

In [1]:
import os
from tqdm import tqdm 
import pickle

# Set the PGDATABASE environment variable
os.environ["PGDATABASE"] = "ernieplus"
import psycopg2

def get_all_mesh_terms():
    all_mesh_set = set()
    
    conn = psycopg2.connect("")
    print("Connected to Postgres successfully")
    cur = conn.cursor()
    sql_query = "select mesh_terms from hm31.pubmed_all_xmls_new am"

    cur.execute(sql_query)

    result = cur.fetchall()
    for idx, row in tqdm(enumerate(result), total = 34780700):
        if len(row) == 0 or len(row[0]) == 0:
            continue

   
        mesh_terms = row[0].split()[0].split(',')
        #print(mesh_terms)
        
        for mesh in mesh_terms:
            all_mesh_set.add(mesh)
            
    return all_mesh_set

# all_mesh_set = get_all_mesh_terms()

# with open('all_mesh.pkl', 'wb') as file:
#     pickle.dump(all_mesh_set, file)

In [2]:
with open('all_mesh.pkl', 'rb') as file:
    all_mesh_set = pickle.load(file)
    
print(len(all_mesh_set))

29944


<h1>Obtaining MeSH tree structure </h1>

In [3]:
import requests
import time
import multiprocessing 
import json

def retrieve_mesh_tree(UI, idx, shared_dict):
    url = f"https://meshb.nlm.nih.gov/record/ui?ui={UI}"
    with requests.Session() as session:
        url = f"https://meshb.nlm.nih.gov/record/ui?ui={UI}"

        response = session.get(url)

    if response.status_code == 200:
        text_of_interest = response.text
        
        try:
            treeNumber_idx = text_of_interest.index('treeNumber_0')
            uniqueID = text_of_interest.index('Unique ID')
            limited_string = text_of_interest[treeNumber_idx:uniqueID]
            end_index = limited_string.find('</a>')
            counter = 1

            while end_index-counter + 1 >= 0:
                if limited_string[end_index-counter] == '>':
                    shared_dict[UI] = limited_string[end_index-counter + 1:end_index]
                    break
                counter += 1
            
        except:
            shared_dict[UI] = "failed"

    else:
        shared_dict[UI] = "failed"
    

temp_mesh = {}
all_mesh_terms = multiprocessing.Manager().dict()

lst = []


for idx, mesh in enumerate(all_mesh_set):  
        if not mesh in all_mesh_terms:
            temp_mesh[mesh] = 1
            

for index, mesh in enumerate(temp_mesh.keys()):
    lst.append((mesh, index, all_mesh_terms))

    
CHUNK_SIZE = 100
SPLIT = len(lst) // CHUNK_SIZE + 1


for i in tqdm(range(SPLIT)):
    if i != SPLIT -1:
        with multiprocessing.Pool(processes=10) as pool:
                pool.starmap(retrieve_mesh_tree, lst[i*CHUNK_SIZE: (i+1) * CHUNK_SIZE])
    
    else:
        with multiprocessing.Pool(multiprocessing.cpu_count()) as pool:
            pool.starmap(retrieve_mesh_tree, lst[i*CHUNK_SIZE: ])  
    
    pool.close()
    pool.join()


        
    time.sleep(10)
    

with open('all_mesh_retrieved', "w") as json_file:
    json_file.write(all_mesh_terms)

  0%|          | 1/300 [00:15<1:14:45, 15.00s/it]


ConnectionError: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))

In [ ]:
mesh_saved = {}

for key,value in all_mesh_terms.items():
    mesh_saved[key] = value
    
    
json_data = json.dumps(mesh_saved, indent=4)  # You can use indent for pretty formatting

# Write the JSON data to the file
with open('all_mesh_retrieved.json', 'w') as json_file:
    json_file.write(json_data)


In [ ]:
!ls
!rm all_mesh_retrieved